In [18]:
from pathlib import Path
import torch
# import torch.backends.cudnn as cudnn
from numpy import random
import sys
from os.path import abspath
from tqdm import tqdm

# Add the new path to the system path
base_path = abspath('./../training/model/yolov7/')
sys.path.append(base_path)

from models.experimental import attempt_load
from utils.datasets import LoadImages
from utils.general import check_img_size, non_max_suppression, scale_coords
from utils.torch_utils import select_device, TracedModel


def detect(opt):
    source, weights, view_img, save_txt, imgsz, trace = opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size, not opt.no_trace

    # Initialize
    device = select_device(opt.device)
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size

    if trace:
        model = TracedModel(model, device, opt.img_size)

    if half:
        model.half()  # to FP16

    # Set Dataloader
    dataset = LoadImages(source, img_size=imgsz, stride=stride)

    # Get names and colors
    # names = model.module.names if hasattr(model, 'module') else model.names

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    old_img_w = old_img_h = imgsz
    old_img_b = 1

    for path, img, im0s, vid_cap in tqdm(dataset):
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Warmup
        if device.type != 'cpu' and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):
            old_img_b = img.shape[0]
            old_img_h = img.shape[2]
            old_img_w = img.shape[3]
            for i in range(3):
                model(img, augment=opt.augment)[0]

        # Inference
        with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
            pred = model(img, augment=opt.augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path        p.name=img.jpg
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} person{'s' * (n > 1)}, "  # add to string
                    print(s)
                
                print(det)
                # for *xyxy, conf, cls in reversed(det):

In [19]:
import argparse

opt = argparse.Namespace(
    weights=['./../training/runs/train/MIAP_person_detection/weights/best.pt'],
    source="./../datasets/HaGRID_test/like",
    img_size=640,
    conf_thres=0.317,
    iou_thres=0.45,
    device='0',
    view_img=False,
    save_txt=False,
    save_conf=False,
    nosave=False,
    classes=None,
    agnostic_nms=False,
    augment=False,
    update=False,
    project='runs/detect',
    name='reducing_detect_func',
    exist_ok=False,
    no_trace=False
)

detect(opt)

YOLOR 🚀 690594c torch 2.3.0+cu121 CUDA:0 (NVIDIA A40, 45618.6875MB)

Model Summary: 208 layers, 6007596 parameters, 0 gradients, 13.0 GFLOPS


Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



  0%|          | 0/100 [00:00<?, ?it/s]

1 person, 
tensor([[2.36000e+02, 5.79000e+02, 1.10600e+03, 1.91800e+03, 7.39746e-01, 0.00000e+00]], device='cuda:0')


  2%|▏         | 2/100 [00:00<00:07, 13.33it/s]

...

1 person, 
tensor([[1.48000e+02, 2.02000e+02, 1.41000e+03, 1.90800e+03, 6.62598e-01, 0.00000e+00]], device='cuda:0')
2 persons, 
tensor([[2.46000e+02, 1.24000e+02, 1.10100e+03, 1.91400e+03, 8.99902e-01, 0.00000e+00],
        [1.05700e+03, 5.81000e+02, 1.30700e+03, 9.16000e+02, 8.30078e-01, 0.00000e+00]], device='cuda:0')
1 person, 
tensor([[9.80000e+01, 6.85000e+02, 1.09700e+03, 1.92000e+03, 5.08789e-01, 0.00000e+00]], device='cuda:0')
1 person, 
tensor([[  4.00000,   4.00000, 478.00000, 638.00000,   0.93164,   0.00000]], device='cuda:0')
1 person, 
tensor([[4.05000e+02, 9.10000e+02, 9.79000e+02, 1.92000e+03, 8.99414e-01, 0.00000e+00]], device='cuda:0')
2 persons, 


 96%|█████████▌| 96/100 [00:06<00:00, 20.55it/s]

tensor([[3.03000e+02, 1.04600e+03, 1.19400e+03, 1.92000e+03, 8.22754e-01, 0.00000e+00],
        [1.02000e+02, 9.62000e+02, 1.56000e+02, 1.07200e+03, 4.33105e-01, 0.00000e+00]], device='cuda:0')
1 person, 
tensor([[5.50000e+02, 6.16000e+02, 1.29600e+03, 1.44000e+03, 7.74902e-01, 0.00000e+00]], device='cuda:0')
1 person, 
tensor([[1.00000e+01, 1.54000e+02, 1.82400e+03, 1.44000e+03, 4.67773e-01, 0.00000e+00]], device='cuda:0')
1 person, 
tensor([[2.49000e+02, 6.73000e+02, 1.13700e+03, 1.92000e+03, 8.15430e-01, 0.00000e+00]], device='cuda:0')


100%|██████████| 100/100 [00:06<00:00, 14.97it/s]

1 person, 
tensor([[5.30000e+01, 2.70000e+02, 1.31600e+03, 1.91700e+03, 7.86133e-01, 0.00000e+00]], device='cuda:0')
1 person, 
tensor([[2.06000e+02, 5.08000e+02, 1.10800e+03, 1.91700e+03, 8.87695e-01, 0.00000e+00]], device='cuda:0')
1 person, 
tensor([[1.11000e+02, 8.38000e+02, 1.33200e+03, 1.92000e+03, 7.14844e-01, 0.00000e+00]], device='cuda:0')


In [ ]:
from utils.general import xyxy2xywh